# Variables that can be used in a SoS step

* **Difficulty level**: easy to intermediate
* **Time need to lean**: 30 minutes or less
* **Key points**:  
  * Steps can access system configurations through variable `CONFIG`, `__version__`
  * Steps can access workflow and step information through variables `step_name`, `step_id`, `workflow_id`, and `master_id`
  * Steps have step and substep specific variables such as `step_input`, `_input`, `_depends`, `_output`
  * Steps can use variables defined locally and in global sections
  * Steps can use variables shared by other steps through `sos_variable`

SoS steps are isolated in the sense that they have access to limited set of variables and by default do not share variables. This tutorial lists all the variables that can be used in a step.

## System variables `SOS_VERSION` and `CONFIG`

<div class="bs-callout bs-callout-primary" role="alert">
    <h4>Variables <code>SOS_VERSION</code> and <code>CONFIG</code></h4>
    <ul>
        <li><code>SOS_VERSION</code> is a string for version of the SoS interpreter</li>  
        <li><code>CONFIG</code> is a dictionary containing content from multiple system configuration files</li>
    </ul>
</div>

SoS provides two system variables, one is `SOS_VERSION`, which is the version of the SoS interpreter, the other one is `CONFIG`, which is a dictionary that contains all the configurations stored in system configuration files, and configurations specified with option `-c` (config).

Variable `SOS_VERSION` is just a string

In [7]:
SOS_VERSION

'0.18.2'

Variable `CONFIG` is a dictionary and can be much more complex. For this particular system, it contains keys `localhost`, `hosts`, and `user_name`,  

In [8]:
CONFIG.keys()

dict_keys(['localhost', 'hosts', 'user_name'])

and you can access values in this dictionary just like any other dictionary:

In [10]:
CONFIG['user_name']

'bpeng1'

Notice that `CONFIG` only contains content from system configuration files `{SOS_DIRECTORY}/site_config.yml`, `~/.sos/hosts.yml`, and `~/.sos/config.yml`. If you have your own configuration in a file in JSON or YMAL format, you can specify it from command line using option `-c`. The content of that file will then be available in variable `CONFIG`.

## Workflow and step variables

SoS defines multiple variables during the execution of a step. The first type of variables provides information about the step, which includes

* **`step_name`**: name of the step
* **`step_id`**: Hash ID of the step, which is determined by the content of the step
* **`workflow_id`**: Hash ID of the workflow in which the step is defined. It would be the ID of the nested workflow if the workflow is nested.
* **`master_id`**: Hash ID of the entire workflow, regardless if the step is defined in a nested workflow.

For example, in the following example, two steps have different step names, step IDs, and workflow IDs because `nested` is a nested workflow, but they share the same `master_id`, which equals to `workflow_id` of the outermost workflow.  

In [1]:
%run
[nested]
print(f'Workflow {workflow_id}: step name={step_name}')
print(f'Workflow {workflow_id}: step id={step_id}')
print(f'Workflow {workflow_id}: workflow id={workflow_id}')
print(f'Workflow {workflow_id}: master id={master_id}')

[default]
print(f'Workflow {workflow_id}: step name={step_name}')
print(f'Workflow {workflow_id}: step id={step_id}')
print(f'Workflow {workflow_id}: workflow id={workflow_id}')
print(f'Workflow {workflow_id}: master id={master_id}')
sos_run('nested')

Workflow 1663b815c3bed9f2: step name=default_0
Workflow 1663b815c3bed9f2: step id=950015e2ddfc10d4
Workflow 1663b815c3bed9f2: workflow id=1663b815c3bed9f2
Workflow 1663b815c3bed9f2: master id=1663b815c3bed9f2
Workflow fecfb67fda2f92df: step name=nested_0
Workflow fecfb67fda2f92df: step id=74bf81da4f5098aa
Workflow fecfb67fda2f92df: workflow id=fecfb67fda2f92df
Workflow fecfb67fda2f92df: master id=1663b815c3bed9f2


These variables can be useful, for example, to save runtime information, as in the example of

In [8]:
%preview -n default_10.log
%run 

[10]
with open(step_name + '.log', 'w') as log:
    log.write(f'Step specific log message saved in {step_name}.log')

> default_10.log (49 B):

Step specific log message saved in default_10.log

If a section defines multiple steps, the step variables can be used to define (slightly) different steps according to which step is executing. For example,

In [9]:
%run human

[human_10, mouse_10]
if 'human' in step_name:
   print("I am dealing with human")
else:
   print("I am dealing with mouse")


I am dealing with human


The second type of variables are step input, output, and dependent targets. They are of type `sos_targets` and will be explained later. 

## Runtime variables for steps and substeps

#### `step_input` and `_input`

In SoS, the `input` statement mostly creates a `step_input` object with provided parameters. That is to say,

```
input: 'a.txt', 'b.txt', group_by=1
```
is almost equivalent to

```
step_input = sos_targets('a.txt', 'b.txt', group_by=1)
```
and we can use `sos_targets` objects directly in an `input` statement in more complicated cases.

## Variables defined in global sections

## Variables shared from other steps

## Further reading

* 